# Language Level Topics

## Strict computation

### `seq`

In [67]:
:i `seq`

seq :: a -> b -> b 	-- Defined in ‘GHC.Prim’
infixr 0 `seq`

In [68]:
seq 1 2

2

In [69]:
[1,2] `seq` [3,4]

Line 1: Redundant seq
Found:
[1, 2] `seq` [3, 4]
Why not:
[3, 4]

[3,4]

So `seq` is similar to the Functor's `const` function (see following example):
```haskell
    ⊥ `seq` b = ⊥
    a `seq` b = b
```
where `⊥` is the [bottom](https://wiki.haskell.org/Bottom) .

In [70]:
[] `const` [3,4]

Line 1: Evaluate
Found:
[] `const` [3, 4]
Why not:
[]

[]

In [71]:
:t undefined

undefined :: forall {a}. a

In [72]:
[] `seq` [3,4]

Line 1: Redundant seq
Found:
[] `seq` [3, 4]
Why not:
[3, 4]

[3,4]

In [73]:
import Data.Void

:i Void

type Void :: *
data Void
  	-- Defined in ‘Data.Void’
instance [safe] Show Void -- Defined in ‘Data.Void’
instance [safe] Eq Void -- Defined in ‘Data.Void’
instance [safe] Ord Void -- Defined in ‘Data.Void’
instance [safe] Semigroup Void -- Defined in ‘Data.Void’
instance [safe] Read Void -- Defined in ‘Data.Void’

In [74]:
() `seq` [3,4]

Line 1: Redundant seq
Found:
() `seq` [3, 4]
Why not:
[3, 4]

[3,4]

So `[]` is not the bottom for `list`.

In [75]:
[1..] `seq` [3,4]

[3,4]

This shows non-terminating is not the bottom for `list`. `undefined` does do the job as it returns `undefined` for "undefine `seq` a", in the same token `Void` is the bottom as well, except the it reports error when executing `undefined` or `Void`.

### `$!`
`$!` is strict operation

In [76]:
:t ($!)

($!) :: forall {a} {b}. (a -> b) -> a -> b

In [77]:
:i ($!)

($!) :: (a -> b) -> a -> b 	-- Defined in ‘GHC.Base’
infixr 0 $!

In [78]:
id $! [1,2]

Line 1: Redundant $!
Found:
id $! [1, 2]
Why not:
[1, 2]Line 1: Redundant $!
Found:
id $! [1, 2]
Why not:
id [1, 2]

[1,2]

In [79]:
(++[3,4]) $! [1,2]

Line 1: Redundant $!
Found:
(++ [3, 4]) $! [1, 2]
Why not:
(++ [3, 4]) [1, 2]

[1,2,3,4]

Interesting information on `seq`: [Why seq is bad](https://stackoverflow.com/questions/12687392/why-is-seq-bad).

### Strict computation: `f $! x = f x`
Strict computation for `f` can be forced by using `f $!`.
```haskell
f $! x  = x `seq` f x
```

In [82]:
id $! [1,2]

Line 1: Redundant $!
Found:
id $! [1, 2]
Why not:
[1, 2]Line 1: Redundant $!
Found:
id $! [1, 2]
Why not:
id [1, 2]

[1,2]

One application of this technique is the definiton of `foldr` based on `foldl`:
```haskell
foldr' :: (a -> b -> b) -> b -> [a] -> b
foldr' f z0 xs = foldl f' id xs z0
  where f' k x z = k $! f x z
```

## Unboxing, lifting, and bottom
For a normal Haskell type, a value can be either fully evaluated, an unevaluated thunk, or the special value $\bot$, pronounced (and sometimes written) “bottom”. The value $\bot$ is a placeholder for a computation that does not succeed. Such a computation could take any of several forms. It could be an infinite loop; an application of error; or the special value undefined.

A type that can contain $\bot$ is referred to as *lifted*. All normal Haskell types are lifted. In practice, this means that we can always write error "eek!" or undefined in place of a normal expression.

This ability to store thunks or $\bot$ comes with a performance cost: it adds an extra layer of indirection. To see why we need this indirection, consider the Word32 type. A value of this type is a full 32 bits wide, so on a 32-bit system, there is no way to directly encode the value $\bot$ within 32 bits. The runtime system has to maintain, and check, some extra data to track whether the value is $\bot$ or not.

An *unboxed* value does away with this indirection. In doing so, it gains performance, but sacrifices the ability to represent a thunk or $\bot$. Since it can be denser than a normal Haskell array, an array of unboxed values is an excellent choice for numeric data and bits. 

### The ST monad
In an imperative language, we would simply modify the elements of the array in place; this will be our approach in Haskell, too.

Haskell provides a special monad, named ST, which lets us work safely with mutable state. Compared to the State monad, it has some powerful added capabilities.
- We can thaw an immutable array to give a mutable array; modify the mutable array in place; and freeze a new immutable array when we are done.
- We have the ability to use *mutable references*. This lets us implement data structures that we can modify after construction, as in an imperative language. This ability is vital for some imperative data structures and algorithms, for which similarly efficient purely functional alternatives have not yet been discovered.

The IO monad also provides these capabilities. The major difference between the two is that the ST monad is intentionally designed so that we can escape from it back into pure Haskell code. We enter the ST monad via the execution function runST, in the same way as for most other Haskell monads (except IO, of course), and we escape by returning from runST.

When we apply a monad's execution function, we expect it to behave repeatably: given the same body and arguments, we must get the same results every time. This also applies to runST. To achieve this repeatability, the ST monad is more restrictive than the IO monad. We cannot read or write files, create global variables, or fork threads. Indeed, although we can create and work with mutable references and arrays, the type system prevents them from escaping to the caller of runST. A mutable array must be frozen into an immutable array before we can return it, and a mutable reference cannot escape at all.